In [1]:
import os, sys
sys.path.insert(0, os.environ['PROJECT_PATH'])

In [45]:
from __future__ import print_function
import pandas as pd, requests, datetime

In [3]:
from config.resources import path_to

In [4]:
funding_rounds_df = pd.read_csv(path_to['with_msa_csv'].format(node='funding_rounds'), encoding='latin1')
funding_rounds_df['announced_on'] = pd.to_datetime(funding_rounds_df['announced_on'])

funding_rounds_df.columns

Index([u'index', u'company_name', u'country_code', u'state_code', u'region',
       u'city', u'investment_type', u'announced_on', u'raised_amount_usd',
       u'raised_amount', u'raised_amount_currency_code',
       u'post_money_valuation_usd', u'post_money_valuation',
       u'post_money_currency_code', u'investor_count', u'cb_url',
       u'company_uuid', u'funding_round_uuid', u'created_at', u'updated_at',
       u'investor_names', u'investor_uuids', u'MSA_NAME', u'MSA_CODE'],
      dtype='object')

## Filtering - by year range and US-based transactions

In [5]:
year_range = list(map(str, range(2010, 2019)))

In [6]:
funding_rounds_us_df = funding_rounds_df[ funding_rounds_df['country_code'] == 'USA' ]
funding_rounds_us_df = funding_rounds_us_df[ funding_rounds_us_df['announced_on'].dt.year.isin(year_range) ]

print(funding_rounds_us_df.shape)

funding_rounds_us_df.head()

(86225, 24)


,index,company_name,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount,...,investor_count,cb_url,company_uuid,funding_round_uuid,created_at,updated_at,investor_names,investor_uuids,MSA_NAME,MSA_CODE
1,1,Celergo,USA,IL,Chicago,Deerfield,series_unknown,2011-03-16,15000000.0,NaN,...,1.0,https://www.crunchbase.com/funding-round/celergo,64ca783a-b434-aaf1-e5d8-62bfd48161c0,e55e06f6-3aae-fbd3-1416-5257e12c967e,2011-03-17 04:07:52,2014-11-06 20:02:49,"{""Frontier Capital""}",{b3f29ace-fef9-a5ee-d1b6-d3a7d25bab81},"Chicago-Naperville-Elgin, IL-IN-WI Metro Area",16980.0
6,6,"DarkOwl, LLC",USA,CO,Denver,Denver,angel,2016-11-29,1866996.0,NaN,...,NaN,https://www.crunchbase.com/funding-round/owl-c...,ebf194e8-6062-4bef-d33f-bf34cfd95189,e55ebd5b-c7a8-158f-159e-cea5fa36c121,2017-05-24 23:51:23,2017-05-24 23:51:23,{},{},"Denver-Aurora-Lakewood, CO Metro Area",19740.0
9,9,Prieto Battery,USA,CO,Fort Collins,Fort Collins,series_unknown,2016-04-15,1159904.0,NaN,...,NaN,https://www.crunchbase.com/funding-round/priet...,06718a3d-afe6-f023-6f54-f26e1d9b5f24,e55f91a3-6824-f27d-deab-8830b2a7081f,2016-04-18 11:12:50,2016-04-18 11:13:23,{},{},"Fort Collins, CO Metro Area",22660.0
10,10,Tagless Style,USA,AR,Little Rock,Little Rock,seed,2015-02-16,250000.0,NaN,...,1.0,https://www.crunchbase.com/funding-round/tagle...,efc81024-d062-9716-4152-c6acf24b81f1,e560596b-cb8e-4d8b-3ee2-fe1a64b1d252,2015-02-17 04:12:29,2015-02-17 04:14:30,{},{},"Little Rock-North Little Rock-Conway, AR Metro...",30780.0
11,11,Safe Shepherd,USA,CA,SF Bay Area,San Francisco,seed,2011-10-26,NaN,NaN,...,1.0,https://www.crunchbase.com/funding-round/safe-...,ac4484c3-c354-0543-4a69-7340847baa69,e560835c-481f-7fee-9ec9-069889454a8e,2014-06-17 06:54:55,2014-06-17 06:54:55,{},{},NaN,NaN


### Focussing directly on the SAN JOSE MSA

In [17]:
path_to_test_dump = os.path.join(os.environ['DATA_PATH'], 'interim', 'funding_rounds', 'sample_funding_rounds.csv')

# msa_codes = [ '45940', '14500', '16820' ] #, '41860', '39340' ]

msa_codes = [ '41940' ] #SAN-JOSE

sample_fr_df = funding_rounds_us_df[ funding_rounds_us_df['MSA_CODE'].isin(msa_codes) ]

sample_fr_df.to_csv(path_to_test_dump, encoding='latin1', index=False)

sample_fr_df.shape

(4126, 24)

### Get a sense of year wise funding round distribution

In [66]:
sample_fr_df['year'] = sample_fr_df['announced_on'].dt.year

sample_fr_by_year = sample_fr_df.groupby('year')
sample_fr_by_year.count()

/home/banerjee_abhimanyu3/.virtualenvs/crunchbase/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,index,company_name,country_code,state_code,region,city,investment_type,announced_on,raised_amount_usd,raised_amount,...,investor_count,cb_url,company_uuid,funding_round_uuid,created_at,updated_at,investor_names,investor_uuids,MSA_NAME,MSA_CODE
year,,,,,,,,,,,,,,,,,,,,,
2010,377,377,377,377,377,377,377,377,350,2,...,198,377,377,377,377,377,377,377,377,377
2011,420,420,420,420,420,420,420,420,348,3,...,270,420,420,420,420,420,420,420,420,420
2012,547,547,547,547,547,547,547,547,449,2,...,417,547,547,547,547,547,547,547,547,547
2013,554,554,554,554,554,554,554,554,446,5,...,402,554,554,554,554,554,554,554,554,554
2014,653,653,653,653,653,653,653,653,504,10,...,467,653,653,653,653,653,653,653,653,653
2015,615,615,615,615,615,615,615,615,452,10,...,466,615,615,615,615,615,615,615,615,615
2016,551,551,551,551,551,551,551,551,390,8,...,416,551,551,551,551,551,551,551,551,551
2017,409,409,409,409,409,409,409,409,303,13,...,339,409,409,409,409,409,409,409,409,409


## Scrape CRUNCHBASE API for the investments

In [19]:
from config.api_specs import fields
from src.data.make_dataset import build_investment_flow_df

path_to_dest = os.path.join(os.path.dirname(path_to_test_dump), 'test_result_data.csv')

build_investment_flow_df(path_to_test_dump, path_to_dest, fields['funding_rounds'])


Preparing to filter dataframe. Please wait ..
Filtering dataframe completed!


Preparing to build investment dataframe. Please wait..
Building investment dataframe completed!. Dumping data to /home/banerjee_abhimanyu3/crunchbase/data/interim/funding_rounds/test_result_data.csv



In [43]:
sample_investments = pd.read_csv(path_to_dest, encoding='latin1')
sample_investments.head()

,index,funding_round_uuid,investment_type,is_lead_investor,money_invested_usd,investee_founded_on,investee_permalink,investee_short_description,investor_founded_on,investor_permalink,investor_short_description,investment_uuid
0,0,e596a90c-80f8-a902-3a9b-34dc60cc115e,series_unknown,NaN,NaN,2006-01-01,contextream,"ConteXtream, a software firm, offers a cloud-b...",2014-01-01,benhamou-global-ventures,First and Most Active Institutional Investors ...,453f252b800241589d2c4c26933f0a8a
1,1,e596a90c-80f8-a902-3a9b-34dc60cc115e,series_unknown,NaN,NaN,2006-01-01,contextream,"ConteXtream, a software firm, offers a cloud-b...",1974-01-01,sofinnova-ventures,"Since 1974, Sofinnova Ventures has partnered w...",c2949dd0806bfd34006a6520a097cfb1
2,0,e5a2919b-5fb9-d5fb-c0ab-04ae841186fa,series_a,True,NaN,2011-01-01,altia-systems,"Altia Systems, a venture-backed company based ...",NaN,naya-ventures,Naya Ventures is a venture company investing i...,7b4605730fc9cadc6bebe579c42c6df8
3,0,e5a363a8-1b25-ac1b-c0ca-9714c76c2637,series_unknown,False,NaN,2012-01-01,splenvid,Splenvid is video storytelling made simple. Ca...,2011-01-01,ffl,Founder Friendly Labs (FFL) is the equity-free...,a76068a549ee9cf1dff920cc31319003
4,0,e5a44cab-869a-0c11-13cd-606e0fba0dae,seed,NaN,NaN,2012-01-01,nimbleheart,NimbleHeart team has multidisciplinary skills ...,2010-01-01,angellist,"AngelList is a U.S. website for startups, ange...",7ea851a261342897f838d22cf8e9b17c


### Checking validity of data

In [47]:
nrows, _ = sample_investments.shape
invalid_money_rows = sum(sample_investments['money_invested_usd'].isnull())

print('Number of rows with valid investment dollars:', nrows - invalid_money_rows)

Number of rows with valid investment dollars: 495


In [52]:
sample_invalid_investments = sample_investments[ sample_investments['money_invested_usd'].isnull() ]
sample_fr_years = sample_fr_df[ ['funding_round_uuid', 'announced_on'] ]
merged_data = pd.merge(sample_invalid_investments, sample_fr_years, how='inner')

merged_data['year'] = merged_data['announced_on'].dt.year

max_year = merged_data['year'].max()
min_year = merged_data['year'].min()

print('Year Range for invalid data: {min_yr} - {max_yr}'.format(min_yr=min_year, max_yr=max_year))

Year Range for invalid data: 2010 - 2017


In [53]:
merged_data[ merged_data['year'] == 2017 ].head()

,index,funding_round_uuid,investment_type,is_lead_investor,money_invested_usd,investee_founded_on,investee_permalink,investee_short_description,investor_founded_on,investor_permalink,investor_short_description,investment_uuid,announced_on,year
35,0,e62a6cdc-e2e8-2d1b-5146-186c6d7fa366,series_d,False,NaN,2007-01-01,emergent-payments,Emergent provides optimized payment solutions ...,2017-01-01,responsible-gold-holdings,Company,eb5ff7e60329563b1bf42412836516f5,2017-03-28,2017
73,0,a29bfe8d-389a-07a6-c4f8-9d11127bd24f,seed,NaN,NaN,2016-12-16,smart-alto,Text messaging robot that helps real estate ag...,2005-03-01,y-combinator,Y Combinator is a startup accelerator based in...,04b1b0fd32387a894650eccfbd7c37fa,2017-01-01,2017
92,0,736b8db8-2b69-14d6-10a2-500ca703968d,series_c,False,NaN,2010-11-01,glooko,Glooko is the Unified Platform for Diabetes Ma...,1919-01-01,mayoclinic,MayoClinic a nonprofit medical practice and me...,2302a7307eea8bb45334dcdfb0d763d1,2017-06-27,2017
93,1,736b8db8-2b69-14d6-10a2-500ca703968d,series_c,False,NaN,2010-11-01,glooko,Glooko is the Unified Platform for Diabetes Ma...,2000-01-01,insulet,Insulet Corporation is engaged in the developm...,285b9bb85c8ac828b717ae540d8738d6,2017-06-27,2017
94,2,736b8db8-2b69-14d6-10a2-500ca703968d,series_c,True,NaN,2010-11-01,glooko,Glooko is the Unified Platform for Diabetes Ma...,2008-01-01,georgian-partners,Georgian Partners is a thesis-driven growth eq...,42eff18950210e9c87fd64fafe4d12c3,2017-06-27,2017
